# 19

## 239. Sliding Window Maximum

Given an array nums, there is a sliding window of size k which is moving from the very left of the array to the very right. You can only see the k numbers in the window. Each time the sliding window moves right by one position. Return the max sliding window.

Follow up: Could you solve it in linear time?

用一個 max heap 存 window 裡的數。當 windown 移動時為了能在 $O(\log k)$ 時間內砍掉 window 最前面的數，保存一個 index table 用來記下每個數字在 heap 裡的 index 為多少，並且每次動到 heap（push 或 remove）時都要去 update 這個 index table。

<font color='red'>下面答案是錯的！</font>

In [51]:
nums = [1,3,-1,-3,5,3,6,7]
k = 3

def maxSlidingWindow(nums, k):    
    heap = []
    for n in nums[:k][::-1]:
        idx = push(heap, n) 
        
    res = [heap[0]]
    for i in range(len(nums)-k):
        remove(heap, idx)
        idx = push(heap, nums[i+k])  # 這個 index 只有加進去的時候是對的。後來多 push 幾個數進 heap 就亂掉了。需要一個能一邊 push 一邊 update index table 的機制
        res.append(heap[0])
        
    return res

maxSlidingWindow(nums, k)

# the right answer is [3,3,5,5,6,7] 

[3, 3, 5, 3, 6, 7]

In [41]:
def push(heap, num):
    '''
    Push one number to a max heap and return the final location (index) of that number. 
    Since list is mutable the heap will be updated after each function call. 
    '''
    idx = len(heap)  # index of num
    heap.append(num)
    isEven = lambda x: x%2==0
    parentIdx = max(idx//2 -1 if isEven(idx) else (idx+1)//2 -1, 0)
    
    # heapify up
    while heap[parentIdx] < heap[idx]:
        heap[parentIdx], heap[idx] = heap[idx], heap[parentIdx]
        idx = parentIdx
        parentIdx = max(idx//2 -1 if isEven(idx) else (idx+1)//2 -1, 0)
        
    return idx

def remove(heap, idx):
    '''
    Remove an element from a max heap by index
    Can not input -1 as index or otherwise children index computation will go wrong. 
    '''
    heap[idx], heap[-1] = heap[-1], heap[idx]
    del heap[-1]
    childIdxL = 2*(idx+1)-1
    childIdxR = 2*(idx+1)
    
    # heapify down
    while (childIdxL < len(heap) and childIdxR < len(heap)) and (heap[idx] < heap[childIdxL] or heap[idx] < heap[childIdxR]): 
        # still has two children
        if heap[childIdxL] < heap[childIdxR]:
            heap[idx], heap[childIdxR] = heap[childIdxR], heap[idx]
            idx = childIdxR
        else:
            heap[idx], heap[childIdxL] = heap[childIdxL], heap[idx]
            idx = childIdxL
        
        childIdxL = 2*(idx+1)-1
        childIdxR = 2*(idx+1)

    if (childIdxL < len(heap)) and (heap[idx] < heap[childIdxL]):   # only has left child
        heap[idx], heap[childIdxL] = heap[childIdxL], heap[idx]

## <font color=red>188. Best Time to Buy and Sell Stock IV</font>

Say you have an array for which the i-th element is the price of a given stock on day i. Design an algorithm to find the maximum profit. You may complete at most k transactions. Note: You may not engage in multiple transactions at the same time (ie, you must sell the stock before you buy again).

最多可以交易 k 次，[這裡](https://www.youtube.com/watch?v=oDhu5uGq_ic)看來的解答。

下面的答案是對的但在 LeetCode submit 不會過。如果遇到很長的 list 或 k >> n 時間或空間會爆。

|  | 2| 5| 7| 1| 4| 3| 1| 3|
|--|--|--|--|--|--|--|--|--|
| 0| 0| 0| 0| 0| 0| 0| 0| 0|
| 1| 0| 3| 5| 5| 5| 5| 5| 5|
| 2| 0| 3| 5| 5| 8| 8| 8| 8|
| 3| 0| 3| 5| 5| 8| 8| 8|10|


$$
T_{i, j} = \max
\begin{cases}
T_{i, j-1} \\
(p_j - p_m) + T_{i-1, m}\quad m=0, 1, \ldots, j-1
\end{cases}, 
$$

可再優化成

\begin{eqnarray*}
T_{i, j} &=& \max
\begin{cases}
T_{i, j-1} \\
p_j + \text{maxDiff}_j
\end{cases}\\\\
\text{maxDiff}_{j} &=& \max(\text{maxDiff}_{j-1},~T_{i-1, j} - p_j), 
\end{eqnarray*}

每一列初始 $\text{maxDiff}_0 = 0 - p_0$。


In [38]:
prices = [2, 5, 7, 1, 4, 3, 1, 3]
k = 3

def maxProfit(k, prices):
    n = len(prices)
    t = [[0]*n for m in range(k+1)]
    
    for i in range(1, k+1):
        maxDiff = -prices[0]
        for j in range(1, n):
            t[i][j] = max(t[i][j-1], prices[j] + maxDiff)
            maxDiff = max(maxDiff, t[i-1][j] - prices[j])

    return t[-1][-1]

maxProfit(k, prices)

10

## 123. Best Time to Buy and Sell Stock III

Say you have an array for which the ith element is the price of a given stock on day i. Design an algorithm to find the maximum profit. You may complete at most two transactions. Note: You may not engage in multiple transactions at the same time (i.e., you must sell the stock before you buy again).

最多可以交易兩次。

Best Time to Buy and Sell Stock (121) 的解答有一個性質就是把 input 整個取負號再倒過來會得到一樣的結果，例如 maxProfit([5, 3, 6, 4]) 會等於 maxProfit([-4, -6, -3, -5])。

可以交易兩次相當於把長度 n 的 array 拆成兩半分別求 121 的 maxProfit 再加起來。有 n 種拆法，例如 [7, 1, 5, 3, 6, 4] = [7, 1] + [5, 3, 6, 4] 是其中一種。隨著分拆點推進，第一個數列變長，第二個數列變短。第一個數列直接丟 121 的解答（但要記下 running max profit，不能用 O(1) space 的作法），第二個數列先倒過來取負號再丟，這樣新增的才是後面的數而不是前面的。

下面 pass1 是在求 [7, 1, 5, 3, 6, 4] 的 121 解答，pass2 是求 [-4, -6, -3, -5, -1, -7] 的 121 解答。最後把其中一個 running max profit 倒過來和另一個相加，得到的數列就是用不同分拆點時得到的兩次交易 max profit。取 max 就是這題的答案。

In [10]:
prices = [7, 1, 5, 3, 6, 4]

def maxProfit(prices):
    n = len(prices)
    
    # pass 1
    runningMax1 = [0]*n
    currentMin = [0]*n
    if n > 0:
        currentMin[0] = prices[0]
        for i, p in zip(range(1, n), prices[1:]):
            runningMax1[i] = max(p-currentMin[i-1], runningMax1[i-1])
            currentMin[i] = min(currentMin[i-1], p)

    # pass 2
    prices = [-p for p in prices[::-1]]
    runningMax2 = [0]*n
    currentMin = [0]*n
    if n > 0:
        currentMin[0] = prices[0]
        for i, p in zip(range(1, n), prices[1:]):
            runningMax2[i] = max(p-currentMin[i-1], runningMax2[i-1])
            currentMin[i] = min(currentMin[i-1], p)

    runningMax1 = [0] + runningMax1
    runningMax2 = runningMax2[::-1] + [0]
        
    return max(m1+m2 for m1, m2 in zip(runningMax1, runningMax2))

maxProfit(prices)

7

## <font color='red'>309. Best Time to Buy and Sell Stock with Cooldown</font>


Say you have an array for which the ith element is the price of a given stock on day i. Design an algorithm to find the maximum profit. You may complete as many transactions as you like (ie, buy one and sell one share of the stock multiple times) with the following restrictions:

* You may not engage in multiple transactions at the same time (ie, you must sell the stock before you buy again).
* After you sell your stock, you cannot buy stock on next day. (ie, cooldown 1 day)
 
賣完之後不能隔天馬上買而是要等一天。


[兩個狀態的 DP](https://youtu.be/Ggzbo9eVrLU?t=92)
：h 和 u 分別為持有股票和空倉時，現金帳戶價值的最大值。把 prices 跑完時 u 的結尾即是答案。Transition function 的 max 裡第一項是第 n 天有改變狀態的情況，第二項是沒有改變狀態。


Base case hold 狀態因為買入股票，現金帳戶變成負值。第二天的 hold 是前兩天股票選便宜的那天買：


|p |8 |6 |4 |3 |3 |2 |3 |5 |8 |3 |8 |2 |6 |
|--|--|--|--|--|--|--|--|--|--|--|--|--|--|
|h|-8|-6|
|u|0|


Transition: 


$$
\begin{pmatrix}
h_n\\
u_n
\end{pmatrix}
=
\begin{pmatrix}
\max(u_{n-2}-p_n, h_{n-1})\\
\max(h_{n-1}+p_n, u_{n-1})
\end{pmatrix}
$$

In [12]:
prices = [8, 6, 4, 3, 3, 2, 3, 5, 8, 3, 8, 2, 6]

def maxProfit(prices):
    n = len(prices)
    if n < 2:
        return 0
    else:
        h = [0]*n
        u = [0]*n
        
        h[:2] = [-prices[0], max(-prices[0], -prices[1])]
        u[1] = max(h[0] + prices[1], u[0])
        
        for i, p in zip(range(2, n), prices[2:]):
            h[i], u[i] = max(u[i-2] - p, h[i-1]), max(h[i-1] + p, u[i-1])
        
        return u[n-1]

maxProfit(prices)

10

## 714. Best Time to Buy and Sell Stock with Transaction Fee

Your are given an array of integers prices, for which the i-th element is the price of a given stock on day i; and a non-negative integer fee representing a transaction fee. You may complete as many transactions as you like, but you need to pay the transaction fee for each transaction. You may not buy more than 1 share of a stock at a time (ie. you must sell the stock share before you buy again.) Return the maximum profit you can make.

用兩個狀態的 DP，同 309，但只需要一天的 Base case。Transition function 改成

$$
\begin{pmatrix}
h_n\\
u_n
\end{pmatrix}
=
\begin{pmatrix}
\max(u_{n-1}-p_n, h_{n-1})\\
\max(h_{n-1}+p_n- f, u_{n-1})
\end{pmatrix}
$$

交易成本一定要加在賣掉那邊（從 hold 轉成 unhold）因為這兩個變數記的是現金帳戶價值。


In [4]:
prices = [8, 6, 4, 3, 3, 2, 3, 5, 8, 3, 8, 2, 6]
fee = 3

def maxProfit(prices, fee):
    n = len(prices)
    if n < 2:
        return 0
    else:
        h = [0]*n
        u = [0]*n
        h[0] = -prices[0]

        for i, p in zip(range(1, n), prices[1:]):
            h[i], u[i] = max(u[i-1] - p, h[i-1]), max(h[i-1] + p - fee, u[i-1])

        return u[n-1]

maxProfit(prices, fee)

6

## 986. Interval List Intersections

Given two lists of closed intervals, each list of intervals is pairwise disjoint and in sorted order. Return the intersection of these two interval lists.

[用 two pointers 分別追蹤 A 和 B 的區間。](https://youtu.be/5xL_N2S3-QU?t=224)兩閉區間有重疊 iff $s \leq e$。如果題目改開區間則是 $s < e$。

In [1]:
A = [[0, 2], [5, 10], [13, 23], [24, 25]]
B = [[1, 5], [8, 12], [15, 24], [25, 26]]

def intervalIntersection(A, B):
    i = j = 0
    res = []
    while i<len(A) and j<len(B):
        s = max(A[i][0], B[j][0])
        e = min(A[i][1], B[j][1])
        if s <= e:
            res.append([s, e])
            
        if A[i][1] < B[j][1]:
            i += 1
        else:
            j += 1
        
    return res
            
intervalIntersection(A, B) 

[[1, 2], [5, 5], [8, 10], [15, 23], [24, 24], [25, 25]]

## 969. Pancake Sorting

Given an array A, we can perform a pancake flip: We choose some positive integer k <= A.length, then reverse the order of the first k elements of A.  We want to perform zero or more pancake flips (doing them one after another in succession) to sort the array A. Return the k-values corresponding to a sequence of pancake flips that sort A.  Any valid answer that sorts the array within 10 * A.length flips will be judged as correct.

[最簡單的算法是 selection sort 變形：](https://zh.wikipedia.org/zh-tw/%E7%85%8E%E9%A4%85%E6%8E%92%E5%BA%8F)每次找出還沒排序的數之中最大的，先把它翻到最前面，再翻到還沒排序的部份的最後面。所以答案的偶數 subarray 會是 n, n-1, ...。看了 LeetCode 最快解法和一些 youtube，大家都只寫這種。


In [2]:
# 假設排序過的 list 是 [1, 2, 3, ..., n]
# 這份 code 成功跑過了 LeetCode 所有的 test case

A = [3,2,4,1]

def pancakeSort(A):
    def pflip(k): 
        A[:k] = A[:k][::-1]
    
    n = len(A)
    res = []
    for k in range(n, 1, -1):
        kIdx = A.index(k)
        if kIdx != 0:
            res.append(kIdx+1)
            pflip(kIdx+1)
            
        res.append(k)
        pflip(k)
        
    return res
        
pancakeSort(A)

[3, 4, 2, 3, 2]

## 200. Number of Islands

Given a 2d grid map of '1's (land) and '0's (water), count the number of islands. An island is surrounded by water and is formed by connecting adjacent lands horizontally or vertically. You may assume all four edges of the grid are all surrounded by water.

每次踩到 1 就 DFS 把同一個島都著色成 0，並且 count 累加一次。

In [3]:
grid = [
    ['1', '1', '0', '0', '0'], 
    ['1', '1', '0', '0', '0'], 
    ['0', '0', '1', '0', '0'],
    ['0', '0', '0', '1', '1']
]

def numIslands(grid):
    if not grid: return 0
    else:
        m = len(grid)
        n = len(grid[0])

        def paint(i, j):
            grid[i][j] = '0'
            if i!=0   and grid[i-1][j]=='1':  paint(i-1, j)
            if i!=m-1 and grid[i+1][j]=='1':  paint(i+1, j)
            if j!=0   and grid[i][j-1]=='1':  paint(i, j-1)
            if j!=n-1 and grid[i][j+1]=='1':  paint(i, j+1)

        count = 0
        for i in range(m):
            for j in range(n):
                if grid[i][j] == '1':
                    paint(i, j)
                    count +=1

        return count

numIslands(grid)

3

## 560. Subarray Sum Equals K

Given an array of integers and an integer k, you need to find the total number of continuous subarrays whose sum equals to k.

用一個 hash table 存下到目前為止，這個 cumsum 一共出現過幾次。

In [4]:
# nums = [-1, -1, 1]
# k = 0

nums = [1, 1, 1]
k = 2

def subarraySum(nums, k):
    cumsumCount = {0: 1}   # running cumsum counts
    curSum = 0
    res = 0
    for i, n in enumerate(nums):
        curSum += n
        res += cumsumCount.get(curSum-k, 0)  # 一定要放在 update 字典之前不然遇到 k=0 的情況會把自己算進去
        
        if curSum in cumsumCount:
            cumsumCount[curSum] += 1
        else:
            cumsumCount[curSum] = 1    
    return res
        
subarraySum(nums, k)

2

## 64. Minimum Path Sum

Given a m x n grid filled with non-negative numbers, find a path from top left to bottom right which minimizes the sum of all numbers along its path. Note: You can only move either down or right at any point in time.

In [5]:
grid = [
  [1,3,1],
  [1,5,1],
  [4,2,1]
]

def minPathSum(grid):
    m = len(grid)
    n = len(grid[0])

    for i in range(1, m):
        grid[i][0] += grid[i-1][0]

    for j in range(1, n):
        grid[0][j] += grid[0][j-1]

    for i in range(1, m):
        for j in range(1, n):
            grid[i][j] += min(grid[i-1][j], grid[i][j-1])

    return grid[m-1][n-1]

minPathSum(grid)

7

## 47. Permutations II

Given a collection of numbers that might contain duplicates, return all possible unique permutations.

如果 input list 裡沒有重覆的數字，答案就是把每一個數字抓出來當開頭，剩下的拿去做 permutation（遞迴）。現在 input list 雖有重覆的數字，但重覆的數字只抓出來當開頭一次就會得到正確答案。例如 [1, 1, 1, 2, 2, 3, 3, 3, 3] 之中 1, 2, 3 各當開頭一次。第一個迴圈只是為了抓每個數字第一次出現時的 index。

In [6]:
nums = [3, 3, 5]

def permuteUnique(nums):
    if not nums:
        return [nums]
    else:
        idx = {}
        for i, n in enumerate(nums):
            if n not in idx:
                idx[n] = i
                
        res = []
        for n, i in idx.items():
            for l in permuteUnique(nums[:i] + nums[i+1:]):
                res.append([n]+l)
                
        return res
    
permuteUnique(nums)

[[3, 3, 5], [3, 5, 3], [5, 3, 3]]

## 15. 3Sum

Given an array nums of n integers, are there elements a, b, c in nums such that a + b + c = 0? Find all unique triplets in the array which gives the sum of zero. The solution set must not contain duplicate triplets.

可以用 two sum (see the 2nd cell below) 但既然都要 $O(n^2)$ 不如[先排序然後用三個指標從左右往中間靠近。](https://www.youtube.com/watch?v=aSVI-DekuTk)

In [7]:
nums = [-1, 0, 1, 2, -1, -4]

def threeSum(nums):
    nums = sorted(nums)
    i = 0
    j = 1
    k = len(nums)-1
    
    res = []
    for i in range(len(nums)-2):
        if i > 0 and nums[i] == nums[i-1]:    # skip duplicate numbers
            continue
            
        j = i+1
        k = len(nums)-1
        while j<k:
            s = nums[i] + nums[j] + nums[k]
            if s > 0:    k -= 1
            elif s < 0:  j += 1
            elif s == 0: 
                res.append([nums[i], nums[j], nums[k]])
                while j < k and nums[j] == nums[j+1]:     # skip duplicate numbers
                    j += 1
                while j < k and nums[k] == nums[k-1]:     # skip duplicate numbers
                    k -= 1
                j += 1
                k -= 1  
    return res

threeSum(nums)

[[-1, -1, 2], [-1, 0, 1]]

In [8]:
# solution making use of two sum

nums = [-1, 0, 1, 2, -1, -4]

def twoSum(nums, target):
    '''
    This is a stronger version of twoSum that can handle multiple solutions as well as no solution
    '''
    allSln = set([])
    s = set([])
    for n in nums:
        if n in s:
            allSln.add((n, target-n))
        s.add(target - n)

    return allSln

def threeSum(nums):
    res = set([])
    for i, n in enumerate(nums):
        for a, b in twoSum(nums[i+1:], -n):
            res.add(tuple(sorted([n, a, b])))

    return [list(sln) for sln in res]

# twoSum([1, 7, 5, 5, 4, 4, 4, 4, 5, 3, 3], 8)
# twoSum([1, 2], 8)

threeSum(nums)

[[-1, 0, 1], [-1, -1, 2]]

## 104. Maximum Depth of Binary Tree

Given a binary tree, find its maximum depth. The maximum depth is the number of nodes along the longest path from the root node down to the farthest leaf node. Note: A leaf is a node with no children.

Example: Given binary tree [3,9,20,null,null,15,7],

```
    3
   / \
  9  20
    /  \
   15   7
```


In [9]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right

def maxDepth(root):
    if root:
        return 1 + max(self.maxDepth(root.left), self.maxDepth(root.right))
    else:
        return 0


## 344. Reverse String

Write a function that reverses a string. The input string is given as an array of characters char[]. Do not allocate extra space for another array, you must do this by modifying the input array in-place with O(1) extra memory. You may assume all the characters consist of printable ascii characters.

In [6]:
s = ["h","e","l","l","o"]

def reverseString(s):
    n = len(s)
    for i in range(n//2):
        s[i], s[n-i-1] = s[n-i-1], s[i]
    return s

reverseString(s)

['o', 'l', 'l', 'e', 'h']

## 122. Best Time to Buy and Sell Stock II

Say you have an array prices for which the ith element is the price of a given stock on day i. Design an algorithm to find the maximum profit. You may complete as many transactions as you like (i.e., buy one and sell one share of the stock multiple times). Note: You may not engage in multiple transactions at the same time (i.e., you must sell the stock before you buy again).

In [7]:
prices = [7, 1, 5, 3, 6, 4]

def maxProfit(prices):
    profit = 0 
    for i in range(1, len(prices)):
        if prices[i] > prices[i-1]:
            profit += (prices[i] - prices[i-1])
    return profit

maxProfit(prices)

7

## 121. Best Time to Buy and Sell Stock

Say you have an array for which the ith element is the price of a given stock on day i. If you were only permitted to complete at most one transaction (i.e., buy one and sell one share of the stock), design an algorithm to find the maximum profit. Note that you cannot sell a stock before you buy one.

直接找出最大和最小再相減行不通，因為可能出現先賣再買。但一邊求 running max profit 一邊 update current min 就不會有這個問題了。

In [9]:
prices = [7, 1, 5, 3, 6, 4]

def maxProfit(prices):
    res = 0
    if prices:
        currentMin = prices[0]
        for p in prices[1:]:
            res = max(p-currentMin, res)
            currentMin = min(currentMin, p)
            
    return res

def maxProfit_(prices):
    '''
    Can use solution of 53 because profit is sum of daily profits from consecutive days
    '''
    return max(0, maxSubArray(np.diff(prices))) if len(prices)>1 else 0

maxProfit(prices)

5

## 53. Maximum Subarray

Given an integer array nums, find the contiguous subarray (containing at least one number) which has the largest sum and return its sum.

In [6]:
nums = [-2, 1, -3, 4, -1, 2, 1, -5, 4]

def maxSubArray(nums):
    maxSum = curMax = nums[0]
    for i  in range(1, len(nums)):
        curMax = max(nums[i], curMax + nums[i])
        maxSum = max(curMax, maxSum)
        
    return maxSum
    
maxSubArray(nums)

6

## 1. Two Sum

Given an array of integers, return indices of the two numbers such that they add up to a specific target. You may assume that each input would have exactly one solution, and you may not use the same element twice.

In [11]:
nums = [2, 7, 11, 5]
target = 9

def twoSum(nums, target):
    d = {}
    for i, n in enumerate(nums):
        if n in d:
            return [d[n], i]
        else:
            d[target - n] = i
        
twoSum(nums, target)

[0, 1]